# Neste trabalho, vamos analisar empréstimos imobiliários
##### Nossa base de dados compõe de um arquivo zipado de 166.13 MB (que dezipado chega a 1,5Gb). Nosso objetivo é prever se um determinado cliente irá dar calote no empréstimo ou não baseado. Vamos usar métodos de Deep Learning para buscar cumprir esse objetivo. Antes disso, devemos preparar nossa base de dados, que é o que faremos inicialmente. O link para a base de dados está aqui https://www.kaggle.com/datasets/deependraverma13/house-loan-data-analysis-deep-learning


In [1]:
# Importação das bibliotecas a serem utilizadas

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from pathlib import Path
import datetime as dt
import time 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# Código extra -> Definição de padrões de fonte nas figuras apresentadas
plt.rc('font', size=12)
plt.rc('axes', labelsize=12, titlesize=12)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# Código extra -> Definição de padrões de visualização dos dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [3]:
# Código extra -> Salvar as figuras no formato .png
IMAGES_PATH = Path() / "images" / "end_to_end_project"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
# Leitura do arquivo
# Caminho para o arquivo zipado
caminho_arquivo_zip = r'C:\Users\danie\OneDrive\Área de Trabalho\Mestrado\Deep Learning\Trabalhoarchive.zip' # O leitor desse documento deverá substituir esse
# caminho por onde escolheu salvar a base de dados

#Nome do arquivo CSV dentro do arquivo zip
nome_arquivo_csv = 'loan_data (1).csv'

# Extração o arquivo CSV do arquivo zip
with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
    # Verificar se o arquivo CSV está presente no zip
    if nome_arquivo_csv in zip_ref.namelist():
        with zip_ref.open(nome_arquivo_csv) as csv_file:
            # Ler o arquivo CSV e armazenar na variável df
            df = pd.read_csv(csv_file,low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\danie\\OneDrive\\Área de Trabalho\\Mestrado\\Deep Learning\\Trabalhoarchive.zip'

In [ ]:
# Não temos informação sobre o significado de FLAGDOCUMENT ou EXTSOURCE, vamos retirar essas colunas para evitr maiores problemas
# Retiramos também a coluna de ID, pois ela não agregará em nada no nosso modelo futuro
df = df[df.columns.drop(list(df.filter(regex='FLAG_DOCUMENT_')))] 
df = df[df.columns.drop(list(df.filter(regex='EXT_SOURCE_')))] 
df = df.drop(['SK_ID_CURR'], axis=1)

## Lidando com dados faltantes

In [ ]:
df.isna().sum(axis=1).value_counts().plot(
    kind="bar", title="Quantidade de valores faltantes", figsize = (21,13)
)
plt.show()

In [ ]:
pd.set_option('display.max_rows', None)
missing = df.isnull().sum()
100*missing/df.shape[0] # Números em porcentagem, para facilitar a visualização

In [ ]:
# Temos várias lacunas grandes no nosso dataset, por escolha, vamos excluir colunas com mais de 30% de dados faltantes.
# Com uma base de dados tão esburacada, nosso modelo não iria performar bem nessa parte.
missing = df.isnull().sum()
for col in missing[missing > 0.3].index.tolist():
    del df[col]

In [ ]:
# Vamos verificar o tipo de dado em cada coluna e quantos dados diferentes temos em cada coluna
for x in df.columns:
    print(f"{x} --> {np.dtype(df[x]), df[x].nunique()}")

### Devemos perceber que, apesar de algumas colunas serem numéricas, elas são puramente categóricas. 

In [ ]:
# Vamos agora separar as colunas que são categóricas das que são puramente numéricas
# Vamos usar 10 valores distintos como ponto de corte para os casos de dúvida
# Nossas colunas numéricas são então
num = ['TARGET','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','REGION_POPULATION_RELATIVE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION',
       'DAYS_ID_PUBLISH','HOUR_APPR_PROCESS_START']

# Nossas colunas de categoria, são as restantes
cat = [col for col in df.columns.values if col not in num]
for col in num:
    df[col] = (df[col].astype(float))

In [ ]:
# Vamos ver algumas métricas estatísticas das colunas numéricas
df[num].describe()

In [ ]:
## Vamos visualizar como alguns dados estão correlacionados com nossa variável de interesse 'TARGET' para 
## facilitar nosso entendimento
corr_matrix = df[num].corr(numeric_only=True)
corr_matrix["TARGET"].abs().sort_values(ascending=False)*100 # Os valores abaixo estão em porcentagem!!!

In [ ]:
# Por simplicidade, vamos separar 5 atributos com maior correlação para uma análise mais profunda
# Vamos selecionar apenas as colunas que possuem correlação maior que 5% em valor absoluto

top5 = ['DAYS_BIRTH','DAYS_ID_PUBLISH','DAYS_EMPLOYED','DAYS_REGISTRATION','REGION_POPULATION_RELATIVE']


In [ ]:
# Análise do top 5
scatter_matrix(df[top5], figsize=(21, 15))
save_fig("scatter_matrix_plot")  # extra
plt.show()

In [ ]:
# A distribuição da variável 'DAYS_EMPLOYED' está um pouco fora do comum, vamos analisá-la mais a fundo
pd.set_option('display.max_rows', 50)
df["DAYS_EMPLOYED"].value_counts()

In [ ]:
# Percebemos no gráfico anterior que possivelmente temos um valor anômalo em torno de 350.000
# Nossa estratégia escolhida para lidar com valores faltantes será de substituição pela média dos outros valores
df['DAYS_EMPLOYED_NAN'] = (df["DAYS_EMPLOYED"]==365243).astype('int')
# Vamos agora trocar esses valores por NaN, e, depois, pela sua média

df['DAYS_EMPLOYED'].replace(365243.0, np.nan, inplace=True)
media = df['DAYS_EMPLOYED'].mean()
df['DAYS_EMPLOYED'].replace(np.nan, media , inplace=True)
# Vamos agora ver como a correlação ficou
df[num].corr(numeric_only = True)['TARGET'].abs().sort_values(ascending=False)*100

In [ ]:
# Nova análise do Top 5
scatter_matrix(df[top5], figsize=(21, 15))
plt.show()

In [ ]:
df[top5].hist(bins=50, figsize=(12, 8))
plt.show()